# Imports

In [31]:
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn import metrics
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
import sqlite3
import csv
import pyodbc

# Database Connectie

In [32]:
sales_con = None
crm_con = None
staff_con = None
GO_SALES_PRODUCT_FORECASTData = None
GO_SALES_INVENTORY_LEVELSData = None



try : 
        sales_con = sqlite3.connect("../Assets/Week 2/go_sales.sqlite")
        print("Connected to go_sales sqlite.")


        crm_con = sqlite3.connect("../Assets/Week 2/go_crm.sqlite")
        print("Connected to go_crm sqlite.")


        staff_con = sqlite3.connect("../Assets/Week 2/go_staff.sqlite")
        print("Connected to go_staff sqlite.")

        GO_SALES_PRODUCT_FORECASTData = pd.read_csv("../Assets/Week 2/GO_SALES_PRODUCT_FORECASTData.csv")
        print("Connected to GO_SALES_PRODUCT_FORECASTData")

        GO_SALES_INVENTORY_LEVELSData = pd.read_csv("../Assets/Week 2/GO_SALES_INVENTORY_LEVELSData.csv", header=0, index_col=False)
        print("Connected to GO_SALES_INVENTORY_LEVELSData.csv")


except sqlite3.Error as error: 
        print("Failed to read data from sqlite table", error)

DataWarehouse_con = None

try : 
        pass
except : 
        pass




Connected to go_sales sqlite.
Connected to go_crm sqlite.
Connected to go_staff sqlite.
Connected to GO_SALES_PRODUCT_FORECASTData
Connected to GO_SALES_INVENTORY_LEVELSData.csv


## GO_CRM database tabellen

In [33]:
# go_crm

global go_crm_age_group
go_crm_age_group = pd.read_sql_query("SELECT * FROM {}".format("age_group"),crm_con)
go_crm_age_group = go_crm_age_group[go_crm_age_group.columns.drop(list(go_crm_age_group.filter(regex="TRIAL")))]


global go_crm_country
go_crm_country = pd.read_sql_query("SELECT * FROM {}".format("country"),crm_con)
go_crm_country = go_crm_country[go_crm_country.columns.drop(list(go_crm_country.filter(regex="TRIAL")))]

global go_crm_retailer
go_crm_retailer = pd.read_sql_query("SELECT * FROM {}".format("retailer"),crm_con)
go_crm_retailer = go_crm_retailer[go_crm_retailer.columns.drop(list(go_crm_retailer.filter(regex="TRIAL")))]

global go_crm_retailer_contact
go_crm_retailer_contact = pd.read_sql_query("SELECT * FROM {}".format("retailer_contact"),crm_con)
go_crm_retailer_contact = go_crm_retailer_contact[go_crm_retailer_contact.columns.drop(list(go_crm_retailer_contact.filter(regex="TRIAL")))]

global go_crm_retailer_headquarters
go_crm_retailer_headquarters = pd.read_sql_query("SELECT * FROM {}".format("retailer_headquarters"),crm_con)
go_crm_retailer_headquarters = go_crm_retailer_headquarters[go_crm_retailer_headquarters.columns.drop(list(go_crm_retailer_headquarters.filter(regex="TRIAL")))]

global go_crm_retailer_segment
go_crm_retailer_segment = pd.read_sql_query("SELECT * FROM {}".format("retailer_segment"),crm_con)
go_crm_retailer_segment = go_crm_retailer_segment[go_crm_retailer_segment.columns.drop(list(go_crm_retailer_segment.filter(regex="TRIAL")))]

global go_crm_retailer_site
go_crm_retailer_site = pd.read_sql_query("SELECT * FROM {}".format("retailer_site"),crm_con)
go_crm_retailer_site = go_crm_retailer_site[go_crm_retailer_site.columns.drop(list(go_crm_retailer_site.filter(regex="TRIAL")))]

global go_crm_retailer_type
go_crm_retailer_type = pd.read_sql_query("SELECT * FROM {}".format("retailer_type"),crm_con)
go_crm_retailer_type = go_crm_retailer_type[go_crm_retailer_type.columns.drop(list(go_crm_retailer_type.filter(regex="TRIAL")))]

global go_crm_sales_demographic
go_crm_sales_demographic = pd.read_sql_query("SELECT * FROM {}".format("sales_demographic"),crm_con)
go_crm_sales_demographic = go_crm_sales_demographic[go_crm_sales_demographic.columns.drop(list(go_crm_sales_demographic.filter(regex="TRIAL")))]

global go_crm_sales_territory
go_crm_sales_territory = pd.read_sql_query("SELECT * FROM {}".format("sales_territory"),crm_con)
go_crm_sales_territory = go_crm_sales_territory[go_crm_sales_territory.columns.drop(list(go_crm_sales_territory.filter(regex="TRIAL")))]

In [34]:
go_crm = [go_crm_age_group,go_crm_country,go_crm_retailer,go_crm_retailer_contact,go_crm_retailer_headquarters,
          go_crm_retailer_segment,go_crm_retailer_site,go_crm_retailer_type,go_crm_sales_demographic,go_crm_sales_territory]

go_crm_age_group["AGE_GROUP_CODE"] = go_crm_age_group["AGE_GROUP_CODE"].astype(int)
go_crm_age_group["LOWER_AGE"] = go_crm_age_group["LOWER_AGE"].astype(int)
go_crm_age_group["UPPER_AGE"] = go_crm_age_group["UPPER_AGE"].astype(int)

go_crm_country["COUNTRY_CODE"] = go_crm_country["COUNTRY_CODE"].astype(int)
go_crm_country["SALES_TERRITORY_CODE"] = go_crm_country["SALES_TERRITORY_CODE"].astype(int)

go_crm_retailer["RETAILER_CODE"] = go_crm_retailer["RETAILER_CODE"].astype(int)
go_crm_retailer["RETAILER_CODEMR"] = go_crm_retailer["RETAILER_CODEMR"].astype(float)
go_crm_retailer["RETAILER_TYPE_CODE"] = go_crm_retailer["RETAILER_TYPE_CODE"].astype(int)

go_crm_retailer_contact["EXTENSION"] = go_crm_retailer_contact["EXTENSION"].astype(float)
go_crm_retailer_contact['RETAILER_CONTACT_CODE'] = go_crm_retailer_contact["RETAILER_CONTACT_CODE"].astype(int)
go_crm_retailer_contact["RETAILER_SITE_CODE"] = go_crm_retailer_contact["RETAILER_SITE_CODE"].astype(int)

go_crm_retailer_headquarters["RETAILER_CODEMR"] = go_crm_retailer_headquarters["RETAILER_CODEMR"].astype(float)
go_crm_retailer_headquarters["COUNTRY_CODE"] = go_crm_retailer_headquarters["COUNTRY_CODE"].astype(int)
go_crm_retailer_headquarters["SEGMENT_CODE"] = go_crm_retailer_headquarters["SEGMENT_CODE"].astype(int)

go_crm_retailer_segment["SEGMENT_CODE"] = go_crm_retailer_segment["SEGMENT_CODE"].astype(int)

go_crm_retailer_site["RETAILER_CODE"] = go_crm_retailer_site["RETAILER_CODE"].astype(int)
go_crm_retailer_site["RETAILER_SITE_CODE"] = go_crm_retailer_site["RETAILER_SITE_CODE"].astype(int)
go_crm_retailer_site["COUNTRY_CODE"] = go_crm_retailer_site["COUNTRY_CODE"].astype(int)
go_crm_retailer_site["ACTIVE_INDICATOR"] = go_crm_retailer_site["ACTIVE_INDICATOR"].astype(int)

go_crm_retailer_type["RETAILER_TYPE_CODE"] = go_crm_retailer_type["RETAILER_TYPE_CODE"].astype(int)

go_crm_sales_territory["SALES_TERRITORY_CODE"] = go_crm_sales_territory["SALES_TERRITORY_CODE"].astype(int)

go_crm_sales_demographic["AGE_GROUP_CODE"] = go_crm_sales_demographic["AGE_GROUP_CODE"].astype(int)
go_crm_sales_demographic["RETAILER_CODEMR"] = go_crm_sales_demographic["RETAILER_CODEMR"].astype(float)


for table in go_crm:
    display(table.dtypes)

AGE_GROUP_CODE    int32
UPPER_AGE         int32
LOWER_AGE         int32
dtype: object

COUNTRY_CODE             int32
COUNTRY_EN              object
FLAG_IMAGE              object
SALES_TERRITORY_CODE     int32
dtype: object

RETAILER_CODE           int32
RETAILER_CODEMR       float64
COMPANY_NAME           object
RETAILER_TYPE_CODE      int32
dtype: object

RETAILER_CONTACT_CODE      int32
RETAILER_SITE_CODE         int32
FIRST_NAME                object
LAST_NAME                 object
JOB_POSITION_EN           object
EXTENSION                float64
FAX                       object
E_MAIL                    object
GENDER                    object
dtype: object

RETAILER_CODEMR    float64
RETAILER_NAME       object
ADDRESS1            object
ADDRESS2            object
CITY                object
REGION              object
POSTAL_ZONE         object
COUNTRY_CODE         int32
PHONE               object
FAX                 object
SEGMENT_CODE         int32
dtype: object

SEGMENT_CODE            int32
LANGUAGE               object
SEGMENT_NAME           object
SEGMENT_DESCRIPTION    object
dtype: object

RETAILER_SITE_CODE     int32
RETAILER_CODE          int32
ADDRESS1              object
ADDRESS2              object
CITY                  object
REGION                object
POSTAL_ZONE           object
COUNTRY_CODE           int32
ACTIVE_INDICATOR       int32
dtype: object

RETAILER_TYPE_CODE     int32
RETAILER_TYPE_EN      object
dtype: object

DEMOGRAPHIC_CODE     object
RETAILER_CODEMR     float64
AGE_GROUP_CODE        int32
SALES_PERCENT        object
dtype: object

SALES_TERRITORY_CODE     int32
TERRITORY_NAME_EN       object
dtype: object

## GO_SALES database tabellen

In [35]:
#go_sales
global product
product = pd.read_sql_query("SELECT * FROM {}".format("product"),sales_con)
product = product[product.columns.drop(list(product.filter(regex="TRIAL")))]


global product_type
product_type = pd.read_sql_query("SELECT * FROM {}".format("product_type"),sales_con)
product_type = product_type[product_type.columns.drop(list(product_type.filter(regex="TRIAL")))]

global product_line
product_line = pd.read_sql_query("SELECT * FROM {}".format("product_line"),sales_con)
product_line = product_line[product_line.columns.drop(list(product_line.filter(regex="TRIAL")))]

global sales_staff
sales_staff = pd.read_sql_query("SELECT * FROM {}".format("sales_staff"),sales_con)
sales_staff = sales_staff[sales_staff.columns.drop(list(sales_staff.filter(regex="TRIAL")))]

global sales_branch
sales_branch = pd.read_sql_query("SELECT * FROM {}".format("sales_branch"),sales_con)
sales_branch = sales_branch[sales_branch.columns.drop(list(sales_branch.filter(regex="TRIAL")))]

global retailer_site
retailer_site = pd.read_sql_query("SELECT * FROM {}".format("retailer_site"),sales_con)
retailer_site = retailer_site[retailer_site.columns.drop(list(retailer_site.filter(regex="TRIAL")))]

global country
country = pd.read_sql_query("SELECT * FROM {}".format("country"),sales_con)
country = country[country.columns.drop(list(country.filter(regex="TRIAL")))]

global order_header
order_header = pd.read_sql_query("SELECT * FROM {}".format("order_header"),sales_con)
order_header = order_header[order_header.columns.drop(list(order_header.filter(regex="TRIAL")))]

global order_method
order_method = pd.read_sql_query("SELECT * FROM {}".format("order_method"),sales_con)
order_method = order_method[order_method.columns.drop(list(order_method.filter(regex="TRIAL")))]

global order_details
order_details = pd.read_sql_query("SELECT * FROM {}".format("order_details"),sales_con)
order_details = order_details[order_details.columns.drop(list(order_details.filter(regex="TRIAL")))]

global returned_item
returned_item = pd.read_sql_query("SELECT * FROM {}".format("returned_item"),sales_con)
returned_item = returned_item[returned_item.columns.drop(list(returned_item.filter(regex="TRIAL")))]

global return_reason
return_reason = pd.read_sql_query("SELECT * FROM {}".format("return_reason"),sales_con)
return_reason = return_reason[return_reason.columns.drop(list(return_reason.filter(regex="TRIAL")))]

global SALES_TARGETData
SALES_TARGETData = pd.read_sql_query("SELECT * FROM {}".format("SALES_TARGETData"),sales_con)
SALES_TARGETData = SALES_TARGETData[SALES_TARGETData.columns.drop(list(SALES_TARGETData.filter(regex="TRIAL")))]

In [36]:
go_sales = [SALES_TARGETData,country,order_details,order_header,
            order_method,product,product_line,product_type,returned_item,
            return_reason, sales_staff,sales_branch,retailer_site]

SALES_TARGETData["Id"] = SALES_TARGETData["Id"].astype(int)
SALES_TARGETData["SALES_STAFF_CODE"] = SALES_TARGETData["SALES_STAFF_CODE"].astype(int)
SALES_TARGETData["SALES_PERIOD"] = SALES_TARGETData["SALES_PERIOD"].astype(int)
SALES_TARGETData["PRODUCT_NUMBER"] = SALES_TARGETData["PRODUCT_NUMBER"].astype(int)
SALES_TARGETData["SALES_TARGET"] = SALES_TARGETData["SALES_TARGET"].astype(int)
SALES_TARGETData["RETAILER_CODE"] = SALES_TARGETData["RETAILER_CODE"].astype(int)
# datatype voor sales_year ?

country["COUNTRY_CODE"] = country["COUNTRY_CODE"].astype(int)

order_details["ORDER_DETAIL_CODE"] = order_details["ORDER_DETAIL_CODE"].astype(int)
order_details["ORDER_NUMBER"] = order_details["ORDER_NUMBER"].astype(int)
order_details["PRODUCT_NUMBER"] = order_details["PRODUCT_NUMBER"].astype(int)
order_details["QUANTITY"] = order_details["QUANTITY"].astype(int)
order_details["UNIT_COST"] = order_details["UNIT_COST"].apply(float)
order_details["UNIT_PRICE"] = order_details["UNIT_PRICE"].apply(float)
order_details["UNIT_SALE_PRICE"] = order_details["UNIT_SALE_PRICE"].apply(float)

order_header["ORDER_NUMBER"] = order_header["ORDER_NUMBER"].astype(int)
order_header["RETAILER_SITE_CODE"] = order_header["RETAILER_SITE_CODE"].astype(int)
order_header["RETAILER_CONTACT_CODE"] = order_header["RETAILER_CONTACT_CODE"].astype(int)
order_header["SALES_STAFF_CODE"] = order_header["SALES_STAFF_CODE"].astype(int)
order_header["SALES_BRANCH_CODE"] = order_header["SALES_BRANCH_CODE"].astype(int)
order_header["ORDER_DATE"] = pd.to_datetime(order_header["ORDER_DATE"])
order_header["ORDER_METHOD_CODE"] = order_header["ORDER_METHOD_CODE"].astype(int)

order_method["ORDER_METHOD_CODE"] = order_method["ORDER_METHOD_CODE"].astype(int)

product["PRODUCT_NUMBER"] = product["PRODUCT_NUMBER"].astype(int)
product["INTRODUCTION_DATE"] = pd.to_datetime(product["INTRODUCTION_DATE"])
product["PRODUCT_TYPE_CODE"] = product["PRODUCT_TYPE_CODE"].astype(int)
product["PRODUCTION_COST"] = product["PRODUCTION_COST"].astype(float)
product["MARGIN"] = product["MARGIN"].astype(float)

product_line["PRODUCT_LINE_CODE"] = product_line["PRODUCT_LINE_CODE"].astype(int)

product_type["PRODUCT_LINE_CODE"] = product_type["PRODUCT_LINE_CODE"].astype(int)
product_type["PRODUCT_TYPE_CODE"] = product_type["PRODUCT_TYPE_CODE"].astype(int)

returned_item["RETURN_CODE"] = returned_item["RETURN_CODE"].astype(int)
returned_item["RETURN_DATE"] = pd.to_datetime(returned_item["RETURN_DATE"], format="%d-%m-%Y %H:%M:%S")
returned_item["ORDER_DETAIL_CODE"] = returned_item["ORDER_DETAIL_CODE"].astype(int)
returned_item["RETURN_REASON_CODE"] = returned_item["RETURN_REASON_CODE"].astype(int)
returned_item["RETURN_QUANTITY"] = returned_item["RETURN_QUANTITY"].astype(int)

return_reason["RETURN_REASON_CODE"] = return_reason["RETURN_REASON_CODE"].astype(int)

sales_staff["SALES_STAFF_CODE"] = sales_staff["SALES_STAFF_CODE"].astype(int)
sales_staff["EXTENSION"] = sales_staff["EXTENSION"].astype(float)
sales_staff["DATE_HIRED"] = pd.to_datetime(sales_staff["DATE_HIRED"])
sales_staff["SALES_BRANCH_CODE"] = sales_staff["SALES_BRANCH_CODE"].astype(int) 

sales_branch["SALES_BRANCH_CODE"] = sales_branch["SALES_BRANCH_CODE"].astype(int)
sales_branch["COUNTRY_CODE"] = sales_branch["COUNTRY_CODE"].astype(int)

retailer_site["RETAILER_SITE_CODE"] = retailer_site["RETAILER_SITE_CODE"].astype(int)
retailer_site["RETAILER_CODE"] = retailer_site["RETAILER_CODE"].astype(int)
retailer_site["COUNTRY_CODE"] = retailer_site["COUNTRY_CODE"].astype(int)
retailer_site["ACTIVE_INDICATOR"] = retailer_site["ACTIVE_INDICATOR"].astype(int)

for table in go_sales:
    display(table.dtypes)
    


C:\Users\Anthony Delgado\AppData\Local\Temp\ipykernel_44432\933722837.py:34: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  product["INTRODUCTION_DATE"] = pd.to_datetime(product["INTRODUCTION_DATE"])
C:\Users\Anthony Delgado\AppData\Local\Temp\ipykernel_44432\933722837.py:54: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sales_staff["DATE_HIRED"] = pd.to_datetime(sales_staff["DATE_HIRED"])


Id                   int32
SALES_STAFF_CODE     int32
SALES_YEAR          object
SALES_PERIOD         int32
RETAILER_NAME       object
PRODUCT_NUMBER       int32
SALES_TARGET         int32
RETAILER_CODE        int32
dtype: object

COUNTRY_CODE      int32
COUNTRY          object
LANGUAGE         object
CURRENCY_NAME    object
dtype: object

ORDER_DETAIL_CODE      int32
ORDER_NUMBER           int32
PRODUCT_NUMBER         int32
QUANTITY               int32
UNIT_COST            float64
UNIT_PRICE           float64
UNIT_SALE_PRICE      float64
dtype: object

ORDER_NUMBER                      int32
RETAILER_NAME                    object
RETAILER_SITE_CODE                int32
RETAILER_CONTACT_CODE             int32
SALES_STAFF_CODE                  int32
SALES_BRANCH_CODE                 int32
ORDER_DATE               datetime64[ns]
ORDER_METHOD_CODE                 int32
dtype: object

...

RETURN_CODE                    int32
RETURN_DATE           datetime64[ns]
ORDER_DETAIL_CODE              int32
RETURN_REASON_CODE             int32
RETURN_QUANTITY                int32
dtype: object

RETURN_REASON_CODE        int32
RETURN_DESCRIPTION_EN    object
dtype: object

SALES_STAFF_CODE              int32
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int32
dtype: object

SALES_BRANCH_CODE     int32
ADDRESS1             object
ADDRESS2             object
CITY                 object
REGION               object
POSTAL_ZONE          object
COUNTRY_CODE          int32
dtype: object

RETAILER_SITE_CODE     int32
RETAILER_CODE          int32
ADDRESS1              object
ADDRESS2              object
CITY                  object
REGION                object
POSTAL_ZONE           object
COUNTRY_CODE           int32
ACTIVE_INDICATOR       int32
dtype: object

# Data samenvoegen

## Retailers en Leeftijdsgroepen

Voor elke retailer staat hieronder hoeveel van hun aankopen bedoeld zijn voor een gegeven leeftijdsgroep.

In [37]:
df = pd.merge(go_crm_sales_demographic, go_crm_age_group, left_on="AGE_GROUP_CODE", how="inner", right_on="AGE_GROUP_CODE")
df = pd.merge(go_crm_retailer_headquarters, df, left_on="RETAILER_CODEMR", how="inner", right_on="RETAILER_CODEMR")
df = pd.merge(go_crm_retailer, df, left_on="RETAILER_CODEMR", how="inner", right_on="RETAILER_CODEMR")
df = df.loc[:,["RETAILER_CODE","RETAILER_CODEMR","RETAILER_NAME","DEMOGRAPHIC_CODE","AGE_GROUP_CODE","SALES_PERCENT","UPPER_AGE","LOWER_AGE"]]


with pd.option_context('display.max_rows', 25,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(df)

# display(df)

,RETAILER_CODE,RETAILER_CODEMR,RETAILER_NAME,DEMOGRAPHIC_CODE,AGE_GROUP_CODE,SALES_PERCENT,UPPER_AGE,LOWER_AGE
0,100,70.0,Golf Masters,644,1,4,20,0
1,100,70.0,Golf Masters,645,2,10,30,21
2,100,70.0,Golf Masters,646,3,16,40,31
3,100,70.0,Golf Masters,647,4,26,50,41
4,100,70.0,Golf Masters,648,5,20,60,51
...,...,...,...,...,...,...,...,...
517,99,69.0,Outdoor Gear Co-op,879,2,13,30,21
518,99,69.0,Outdoor Gear Co-op,880,3,34,40,31
519,99,69.0,Outdoor Gear Co-op,881,4,31,50,41
520,99,69.0,Outdoor Gear Co-op,882,5,7,60,51


In [38]:
df = df.loc[:, [ "RETAILER_NAME","AGE_GROUP_CODE","SALES_PERCENT"]]

df1 = df.loc[(df["AGE_GROUP_CODE"]==1),["SALES_PERCENT"]]
df2 = df.loc[(df["AGE_GROUP_CODE"]==2),["SALES_PERCENT"]]
df3 = df.loc[(df["AGE_GROUP_CODE"]==3),["SALES_PERCENT"]]
df4 = df.loc[(df["AGE_GROUP_CODE"]==4),["SALES_PERCENT"]]
df5 = df.loc[(df["AGE_GROUP_CODE"]==5),["SALES_PERCENT"]]
df6 = df.loc[(df["AGE_GROUP_CODE"]==6),["SALES_PERCENT"]]

# df = df.groupby("AGE_GROUP_CODE").aggregate(["min","max","mean"])

df6.to_csv("age_group_1_SALES_PERCENT", index=False)

display(df6)



,SALES_PERCENT
5,24
11,19
17,17
23,4
29,1
...,...
497,23
503,3
509,2
515,2


## Omzet per retailer per jaar

In [39]:
df2 = pd.merge(order_header, order_details, left_on="ORDER_NUMBER", how="inner", right_on="ORDER_NUMBER")
df2 = pd.merge(go_crm_retailer, df2, left_on="COMPANY_NAME", how='inner', right_on="RETAILER_NAME")

df2 = pd.DataFrame(df2.groupby(["RETAILER_NAME", df2["ORDER_DATE"].dt.year])["UNIT_SALE_PRICE"].sum())

# mask = (df2["ORDER_DATE"].dt.year == 2020)
# df2020 = df2.loc[mask,:]
# df2020["TURNOVER"] = df2020["QUANTITY"] * df2020["UNIT_SALE_PRICE"]
# df2020 = df2020.loc[:,["RETAILER_NAME", "TURNOVER"]]
# df2020 = pd.DataFrame(df2020.groupby("RETAILER_NAME")["TURNOVER"].sum())


# mask = (df2["ORDER_DATE"].dt.year == 2021)
# df2021 = df2.loc[mask,:]
# df2021["TURNOVER"] = df2021["QUANTITY"] * df2021["UNIT_SALE_PRICE"]
# df2021 = df2021.loc[:,["RETAILER_NAME", "TURNOVER"]]
# df2021 = pd.DataFrame(df2021.groupby("RETAILER_NAME")["TURNOVER"].sum())



# mask = (df2["ORDER_DATE"].dt.year == 2022)
# df2022 = df2.loc[mask,:]
# df2022["TURNOVER"] = df2022["QUANTITY"] * df2022["UNIT_SALE_PRICE"]
# df2022 = df2022.loc[:,["RETAILER_NAME", "TURNOVER"]]
# df2022 = pd.DataFrame(df2022.groupby("RETAILER_NAME")["TURNOVER"].sum())

with pd.option_context('display.max_rows', 25,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(df2)

# display(df2021)



# # with pd.ExcelWriter("2020_Turnover.xlsx") as excel_writer :
# #     df2020.to_excel(excel_writer, sheet_name="Sheet1",index=False)

# df2020.to_csv("2020_Turnover.csv")
# df2021.to_csv("2021_Turnover.csv")
# df2022.to_csv("2022_Turnover.csv")


UNIT_SALE_PRICE
RETAILER_NAME     ORDER_DATE                 
Act'N'Up Fitness  2020               25538.79
                  2021               20324.50
                  2022               16785.28
ActiForme         2020                1812.08
                  2021                1583.42
...                                       ...
Weitblick         2021                 433.74
                  2022                 299.99
Weston Outfitters 2020               34799.45
                  2021               37716.65
                  2022               41952.87

[295 rows x 1 columns]

In [54]:
df = pd.read_csv("Enquete.csv")
df = df.convert_dtypes()

print(df.columns)
# print(df.dtypes)

kolom = "Type_Gekocht_Product"

df1 = df.loc[:,[kolom]]

df1[kolom] = df1[kolom].apply(lambda x : x.replace(", ",";"))

df1.to_csv("Enquete resultaat", index=False)


display(df1)

Index(['Bekend_met_Great_Outdoors', 'Leeftijd', 'Frequentie_Streetwear',
       'Aankoopmethode', 'Criteria_Streetwear', 'Uitgaven_Streetwear',
       'Type_Kledingstuk', 'Bron_Trends', 'Waarschijnlijkheid_Gebruik',
       'Suggesties_Verbetering', 'Recente_Aankoop', 'Type_Gekocht_Product'],
      dtype='object')


,Type_Gekocht_Product
0,Campinguitrusting;Visspullen;Geen aankoop gedaan
1,Casual kleding;Schoeisel
2,Buitenkleding
3,Accessoires
4,Casual kleding
5,Geen aankoop gedaan;Accessoires
6,Visspullen;Accessoires
7,Casual kleding;Accessoires
8,Geen aankoop gedaan
9,Schoeisel;Casual kleding
